In [ ]:
"""
Personal Tetris Implementaiton
"""

#TODO: Find some way to render game

# import numpy as np
import random
import sys
import keyboard
import copy

class Tetris:

    # Tetris game class
    config = {
        "rows": 14,
        "cols": 6, # Min 6
        "render": True
    }

    # All pieces at all rotations
    tetris_pieces = {
        0: { # I
            0: [(0,0), (1,0), (2,0), (3,0)],
            90: [(1,0), (1,1), (1,2), (1,3)],
            180: [(3,0), (2,0), (1,0), (0,0)],
            270: [(1,3), (1,2), (1,1), (1,0)],
        },
        1: { # T
            0: [(1,0), (0,1), (1,1), (2,1)],
            90: [(0,1), (1,2), (1,1), (1,0)],
            180: [(1,2), (2,1), (1,1), (0,1)],
            270: [(2,1), (1,0), (1,1), (1,2)],
        },
        2: { # L
            0: [(1,0), (1,1), (1,2), (2,2)],
            90: [(0,1), (1,1), (2,1), (2,0)],
            180: [(1,2), (1,1), (1,0), (0,0)],
            270: [(2,1), (1,1), (0,1), (0,2)],
        },
        3: { # J
            0: [(1,0), (1,1), (1,2), (0,2)],
            90: [(0,1), (1,1), (2,1), (2,2)],
            180: [(1,2), (1,1), (1,0), (2,0)],
            270: [(2,1), (1,1), (0,1), (0,0)],
        },
        4: { # Z
            0: [(0,0), (1,0), (1,1), (2,1)],
            90: [(0,2), (0,1), (1,1), (1,0)],
            180: [(2,1), (1,1), (1,0), (0,0)],
            270: [(1,0), (1,1), (0,1), (0,2)],
        },
        5: { # S
            0: [(2,0), (1,0), (1,1), (0,1)],
            90: [(0,0), (0,1), (1,1), (1,2)],
            180: [(0,1), (1,1), (1,0), (2,0)],
            270: [(1,2), (1,1), (0,1), (0,0)],
        },
        6: { # O/Square
            0: [(1,0), (2,0), (1,1), (2,1)],
            90: [(1,0), (2,0), (1,1), (2,1)],
            180: [(1,0), (2,0), (1,1), (2,1)],
            270: [(1,0), (2,0), (1,1), (2,1)],
        }
    }
    
    def __init__(self):
        self.restart()

    def restart(self):
        # clear and setup board

        # BOARD SETUP
        # 0 = Empty space
        # 1 = Piece
        self.cols = self.config["cols"]
        self.rows = self.config["rows"]
        self.board = [[0] * self.cols  for _ in range(self.rows)]
        self.score = 0
        self.current_piece = 0
        self.next_piece = random.randint(0,6)
        self.pos = [0, int((self.config["cols"] / 2) - 2)] # Middle of top of board
        self.rotation = 0

    def _clear_lines(self):
        # Check for full lines, clear if needed
        lines_full = []
        
        # Get full lines
        for i, row in board:
            if 0 not in row:
                lines_full.append(i)
        
        # Create new board without full lines
        board = [row for index, row in enumerate(board) if index not in lines_full]
        
        # Insert new lines at top
        for line in lines_full:
            board.insert(0, [0 for _ in range(self.cols)])

        # Give score based on lines cleared
        if (len(lines_full) == 4): #TETRIS (extra points)
            score += 8
        else:
            score += len(lines_full)

    # Check for collisions with pieces on board or side of board
    def _is_colliding(self):
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            if x not in range(self.cols) \
            or y not in range(self.rows) \
            or self.board[y][x] == 1:
                return True
        return False 
            
    def _add_piece(self):
        piece = self.tetris_pieces[self.current_piece][self.rotation]

        for x, y in piece:
            x += self.pos[0]
            y += self.pos[1]

            self.board[y][x] = 1

        # add new piece to board and check for collisions
        self.current_piece = self.next_piece
        self.next_piece = random.randint(0,6)

        self.pos = [0, int((self.cols / 2) - 2)]
        self.rotation = 0

        if self._is_colliding():
            # Game over
            self._game_over()


    def _game_over(self):
        #with open("scores.txt", "w") as f:
        #    f.write(self.score)  
        #sys.exit()
        self.restart()

    # Angle is 90 or -90
    def _rotate(self, angle):
        self.rotation += angle

        if self.rotation == 360:
            self.rotation = 0
        elif self.rotation < 0:
            self.rotation = 360

    def step(self, action):
        if (action == "COUNTERCLOCKWISE"):
            self._rotate(-90)
        elif(action == "CLOCKWISE"):
            self._rotate(90)
        elif(action == "RIGHT"):
            self.pos[0] += 1
        elif(action == "LEFT"):
            self.pos[0] -= 1

        # Check collisions and reverse action if not allowed
        if self._is_colliding():
            print("INVALID MOVE")
            # Reverse OP
            if (action == "COUNTERCLOCKWISE"):
                self._rotate(90)
            elif(action == "CLOCKWISE"):
                self._rotate(-90)
            elif(action == "RIGHT"):
                self.pos[0] -= 1
            elif(action == "LEFT"):
                self.pos[0] += 1
        
        # Drop piece down by one and check collision, add to board if colliding
        self.pos[1] += 1
        if self._is_colliding():
            print("PIECE PLACED")
            self.pos[1] -= 1
            self._add_piece()

        if (self.config["render"]):
            self._render()

    def _render(self):
        # Add current piece to temp board
        tempboard = copy.deepcopy(self.board)

        for x, y in self.tetris_pieces[self.current_piece][self.rotation]:
            x += self.pos[0]
            y += self.pos[1]

            tempboard[y][x] = 1

        for row in tempboard:
            print(row)
        print("")
        


In [ ]:
tetris = Tetris()
userinput = ""

while (userinput != "STOP"):
    userinput = input()

    match userinput:
        case "r":
            print("RIGHT")
            tetris.step("RIGHT")
        case "l":
            print("LEFT")
            tetris.step("LEFT")
        case "cc":
            print("CC")
            tetris.step("COUNTERCLOCKWISE")
        case "c":
            print("C")
            tetris.step("CLOCKWISE")
        case "n":
            print("N")
            tetris.step("NOOP")